<a href="https://colab.research.google.com/github/YeongEunLee/XAI_/blob/main/%EB%B2%94%EC%A3%BC%ED%98%95%EC%97%85%EB%8D%B0%EC%9D%B4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pymysql
import numpy as np
import pandas as pd

In [ ]:
!pip install pymysql

     |████████████████████████████████| 51kB 3.1MB/s 


In [ ]:
# MySQL 연결
conn = pymysql.connect(
    user = 'oeueoo',
    passwd = 'whdgkqtjfrP123!',
    host = 'ku-traffic-server.crob43mnux9m.us-east-2.rds.amazonaws.com',
    db = 'KU',
    charset = 'utf8'
)
curs = conn.cursor()

In [ ]:
accident_merge_final = "select * from accident_merge_final"
curs.execute(accident_merge_final)
accident_merge_final = curs.fetchall()

In [ ]:
#accident final df 코드
accident_merge_final = pd.DataFrame(accident_merge_final, columns=['인덱스', '사고일자', '사고시간', '요일', '시군구', '도로명', '사고내용', '사고비용', '사고수', '최소도로폭', '최대도로폭', '평균도로폭', '도로길이', '최고속도', '평균속도', '최저속도', '통행량', '상가수', '상업지역수', '주거지역수', '학교수', '어린이보호구역수', '버스정류장수', '횡단보도수', '평균기온', '일강수량', '노면상태', '기상상태'])
accident_merge_final

,인덱스,사고일자,사고시간,요일,시군구,도로명,사고내용,사고비용,사고수,최소도로폭,최대도로폭,평균도로폭,도로길이,최고속도,평균속도,최저속도,통행량,상가수,상업지역수,주거지역수,학교수,어린이보호구역수,버스정류장수,횡단보도수,평균기온,일강수량,노면상태,기상상태
0,0,20170101,0,일요일,서울특별시 용산구 한남동,독서당로,경상사고,24299000,74,0.0,25.0,18.59140,7316.0,33.0,29.0,25.0,NaN,735,3,14,5,5,46,65,2.7,0.0,0,0
1,1,20170101,0,일요일,서울특별시 강북구 미아동,오현로,중상사고,96861700,25,1.0,20.0,14.92190,3292.0,33.0,25.0,19.0,NaN,131,0,6,3,2,29,33,2.7,0.0,0,0
2,2,20170101,0,일요일,서울특별시 동작구 상도동,양녕로,경상사고,4859800,94,0.0,40.0,18.71680,6367.0,46.0,29.0,12.0,1114.0,305,0,2,0,0,28,54,2.7,0.0,0,0
3,3,20170101,0,일요일,서울특별시 중랑구 면목동,면목로,중상사고,67702900,101,0.0,14.0,10.03750,7198.0,28.0,22.0,16.0,NaN,742,0,3,2,1,41,76,2.7,0.0,0,0
4,4,20170101,0,일요일,서울특별시 서초구 서초동,강남대로61길,경상사고,4859800,15,20.0,20.0,20.00000,320.0,NaN,NaN,NaN,NaN,126,0,1,0,0,1,14,2.7,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98110,98110,20191231,22,화요일,서울특별시 중랑구 면목동,용마산로94길,부상신고사고,2427300,6,2.0,15.0,6.02502,3197.0,NaN,NaN,NaN,NaN,66,0,2,0,0,7,0,-7.9,0.0,0,0
98111,98111,20191231,22,화요일,서울특별시 강남구 삼성동,봉은사로,경상사고,4859800,163,4.0,35.0,31.35290,4760.0,29.0,21.0,14.0,NaN,696,0,1,0,0,27,115,-7.9,0.0,0,0
98112,98112,20191231,22,화요일,서울특별시 송파구 송파동,송이로,경상사고,4859800,44,3.0,42.0,39.63740,2868.0,28.0,20.0,13.0,NaN,151,0,2,3,1,11,60,-7.9,0.0,0,0
98113,98113,20191231,23,화요일,서울특별시 강남구 역삼동,봉은사로,부상신고사고,4854600,163,4.0,35.0,31.35290,4760.0,29.0,22.0,10.0,NaN,696,0,1,0,0,27,115,-7.9,0.0,0,0


## 범주형 data mysql에 새로 추가

In [ ]:
from google.colab import drive
drive.mount('/content/dricve')

Mounted at /content/dricve


In [ ]:
df = pd.read_csv('/content/dricve/MyDrive/Colab Notebooks/accident_roadname_final4.csv')
df

,사고일자,사고시간,사고시간대,요일,weekday,시군구,도로명,노면상태,기상상태
0,2017-01-01,0,1,일요일,1,서울특별시 용산구 한남동,독서당로,0,0
1,2017-01-01,0,1,일요일,1,서울특별시 강북구 미아동,오현로,0,0
2,2017-01-01,0,1,일요일,1,서울특별시 동작구 상도동,양녕로,0,0
3,2017-01-01,0,1,일요일,1,서울특별시 중랑구 면목동,면목로,0,0
4,2017-01-01,0,1,일요일,1,서울특별시 서초구 서초동,강남대로61길,0,0
...,...,...,...,...,...,...,...,...,...
98110,2019-12-31,22,1,화요일,0,서울특별시 중랑구 면목동,용마산로94길,0,0
98111,2019-12-31,22,1,화요일,0,서울특별시 강남구 삼성동,봉은사로,0,0
98112,2019-12-31,22,1,화요일,0,서울특별시 송파구 송파동,송이로,0,0
98113,2019-12-31,23,1,화요일,0,서울특별시 강남구 역삼동,봉은사로,0,0


In [ ]:
accident_merge_final['노면상태'] = df['노면상태']
accident_merge_final['기상상태'] = df['기상상태']

###### 위의 코드까지 실행한 후, accident_merge_final을 보면 범주형 + 연속형 자료가 된다. 해당 데이터 프레임중 필요한 열만 골라 logistic regression을 돌려도 된다.

In [ ]:
category_df = accident_merge_final[['노면상태', '기상상태', '인덱스']]
category_df

,노면상태,기상상태,인덱스
0,0,0,0
1,0,0,1
2,0,0,2
3,0,0,3
4,0,0,4
...,...,...,...
98110,0,0,98110
98111,0,0,98111
98112,0,0,98112
98113,0,0,98113


In [ ]:
# 컬럼의 값과 조건을 비교합니다.
# 그 결과를 새로운 변수에 할당합니다.
is_road = category_df['노면상태'] == 1

# 컬럼의 값과 조건을 비교합니다.
# 그 결과를 새로운 변수에 할당합니다.
is_weather = category_df['기상상태'] == 1

# 두가지 조건를 동시에 충족하는 데이터를 필터링하여 새로운 변수에 저장합니다. (OR)
subset_df = category_df[is_road | is_weather]

# 결과를 출력합니다.
subset_df

,노면상태,기상상태,인덱스
23,1,0,23
24,1,0,24
28,1,1,28
29,1,0,29
30,1,1,30
...,...,...,...
98039,0,1,98039
98070,0,1,98070
98079,1,0,98079
98085,1,0,98085


In [ ]:
array = subset_df.to_numpy()
update_data = array.tolist()
update_data

In [ ]:
len(update_data)

13127

## accident_merge와 accident_merge_final의 bus 데이터 업데이트

In [ ]:
# update_data = ['[버스정류장수,도로명], [,]형태의 배열']]
update_sql = "UPDATE accident_merge_final SET 노면상태=%s, 기상상태=%s WHERE 인덱스=%s"
curs.executemany(update_sql, update_data)
conn.commit()
conn.close()